In [11]:
import pandas as pd

from data import Dataset, get_transfer_dataset
from learner import PredictorLearner, TransferLearner
import analysis as a

# initialise 
rq1_results_fields = ['mse_accuracy_src_no_cv', 'mape_accuracy_src_no_cv', 'mse_accuracy_src_cv', 'mape_accuracy_src_cv',
                     'mse_accuracy_tgt_no_cv', 'mape_accuracy_tgt_no_cv', 'mse_accuracy_tgt_cv', 'mape_accuracy_tgt_cv',
                     'mse_accuracy_trans_no_cv', 'mape_accuracy_trans_no_cv', 'mse_accuracy_trans_cv', 'mape_accuracy_trans_cv']
rq1_results = pd.DataFrame(columns=rq1_results_fields)
rq2_results_fields = ['mse_accuracy_20pct', 'mse_accuracy_40pct', 'mse_accuracy_60pct', 'mse_accuracy_80pct',
                     'mape_accuracy_20pct', 'mape_accuracy_40pct', 'mape_accuracy_60pct', 'mape_accuracy_80pct']
rq2_results = pd.DataFrame(columns=rq2_results_fields)
rq3_results_fields = ['training_time_pred_20pct_no_cv', 'training_time_pred_40pct_no_cv', 'training_time_pred_60pct_no_cv', 
                     'training_time_pred_80pct_no_cv', 'training_time_pred_20pct_cv', 'training_time_pred_40pct_cv', 
                     'training_time_pred_60pct_cv', 'training_time_pred_80pct_cv', 'training_time_tran_20pct_no_cv', 
                     'training_time_tran_40pct_no_cv', 'training_time_tran_60pct_no_cv', 'training_time_tran_80pct_no_cv', 
                     'training_time_tran_20pct_cv', 'training_time_tran_40pct_cv', 'training_time_tran_60pct_cv', 
                     'training_time_tran_80pct_cv']
rq3_results = pd.DataFrame(columns=rq3_results_fields)

dataset_nodejs = Dataset('../dataset/nodejs/43/buffer1.csv', 'nodejs')
dataset_poppler = Dataset('../dataset/poppler/12/cuda.csv', 'poppler')
dataset_x264_src = Dataset('../dataset/x264/43/original_videos_Animation_480P_Animation_480P-087e.csv', 'x264')
dataset_x264_target = Dataset('../dataset/x264/44/original_videos_Animation_480P_Animation_480P-087e.csv', 'x264')
dataset_xz = Dataset('../dataset/xz/23/dickens.csv', 'xz')

In [8]:
### RQ1

predictor_errors = []
transfer_errors = []

X_train, X_validate, y_train, y_validate = dataset_x264_target.get_split_dataset()
predictor = PredictorLearner()
optimised_model, _ = predictor.get_optimal_params(X_validate, y_validate)
predictor.fit(X_train, y_train, premade_model=optimised_model)
predictor_errors.append(predictor.get_error(X_train, y_train))

X_train, X_validate, y_train, y_validate = get_transfer_dataset(dataset_x264_src, dataset_x264_target)
transferrer = TransferLearner()
optimised_model, _ = transferrer.get_optimal_params(X_validate, y_validate)
transferrer.fit(X_train, y_train, premade_model=optimised_model)
transfer_errors.append(transferrer.get_error(X_train, y_train))

print(predictor_errors)
print(transfer_errors)

[array([-0.10863496, -0.26705976, -0.26830512, -0.16817388, -0.36554945])]
[array([-0.44327345, -0.5046296 , -0.380163  , -0.47154408, -0.45512598])]


In [9]:
### RQ3

predictor_training_times_no_cv = []
transfer_training_times_no_cv = []
predictor_training_times = []
transfer_training_times = []

# get 80/20 split dataset
X_train, X_validate, y_train, y_validate = dataset_x264_target.get_split_dataset()
predictor = PredictorLearner()
# optimise model using grid search cross validation
optimised_model, _ = predictor.get_optimal_params(X_validate, y_validate)
predictor.fit(X_train, y_train, premade_model=optimised_model)
# record results
predictor_training_times_no_cv.append(predictor.get_training_time())
predictor_training_times.append(predictor.get_training_time(include_optimisation_time=True))

# get 80/20 split dataset
X_train, X_validate, y_train, y_validate = get_transfer_dataset(dataset_x264_src, dataset_x264_target)
transferrer = TransferLearner()
# optimise model using grid search cross validation
optimised_model, _ = transferrer.get_optimal_params(X_validate, y_validate)
transferrer.fit(X_train, y_train, premade_model=optimised_model)
# record results
transfer_training_times_no_cv.append(transferrer.get_training_time())
transfer_training_times.append(transferrer.get_training_time(include_optimisation_time=True))

print(predictor_training_times)
print(transfer_training_times)
print(predictor_training_times_no_cv)
print(transfer_training_times_no_cv)

[0.20999813079833984]
[0.0]
[0.0009999275207519531]
[0.0]
